# The Basis Mixer!

The previous notebooks explored the individual components of the Basis Function Models. In this notebook we will put everything together to render an expressive performance of a piece given its score.

In [11]:
import numpy as np
from partitura import save_performance_midi, load_musicxml
from partitura.score import expand_grace_notes

from basismixer.performance_codec import get_performance_codec
from basismixer.basisfunctions import make_basis

from helper import load_model

## 0. Select a piece and trained models
We start by defining the piece we want to render (here is an example MusicXML file, but you can set your own!)

In [2]:
xml_fn = './data_examples/Chopin_op10_no3.musicxml'

Now we set the path to the directory of the trained models from our previous notebook (or any other trained models). Following the directory structure from the previous notebook, each

In [3]:
models_dir = './vienna4x22_models/'

## 1. Load the Predictive Model

The function `load_model` looks for the configuration files and the saved model parameters in `models_dir`. This function will return an instance of `FullPredictiveModel`, which is a conveniece *meta-model* which contains a list of the individual models (in `model.models`).

In [4]:
model = load_model(models_dir)
print(model)

FullPredictiveModel(
  (models): ModuleList(
    (0): RecurrentModel(
      (rnn): GRU(35, 128, batch_first=True, bidirectional=True)
      (dense): Linear(in_features=256, out_features=64, bias=True)
      (output): Linear(in_features=64, out_features=4, bias=True)
    )
    (1): FeedForwardModel(
      (hidden_layers): Sequential(
        (0): Linear(in_features=35, out_features=128, bias=True)
        (1): ReLU()
      )
      (output): Linear(in_features=128, out_features=3, bias=True)
    )
  )
)


## 2. Computing the Score Representation.

We load the input MusicXML file specified above and compute the input score representation using the Basis Functions specified by the model (in `model.input_names`).

In [5]:
# Load MusicXML file
part = load_musicxml(xml_fn, force_note_ids=True)
onsets = part.note_array['onset']
expand_grace_notes(part)

# Compute basis functions
_basis, bf_names = make_basis(part, list(set([bf.split('.')[0] for bf in model.input_names])))
basis_idx = np.array([int(np.where(model.input_names == bf)[0]) for bf in bf_names])
basis = np.zeros((len(_basis), len(model.input_names)))
basis[:, basis_idx] = _basis

## 3. Make Predict Performance

We make predictions of the performance using the models. , The `predict` method of `FullPredictiveModel` combines the predictions of the individual predictive models into a single output structured array, where the field names are the output parameters (specified in `model.output_names`).

In [6]:
preds = model.predict(basis, onsets)

print('Field names:', preds.dtype.names)


Field names: ('articulation_log', 'beat_period_mean', 'beat_period_standardized', 'beat_period_std', 'timing', 'velocity_dev', 'velocity_trend')


We can post-process the model predictions

In [7]:
# Post-processing
if 'beat_period_mean' in preds.dtype.names:
    preds['beat_period_mean'] = np.ones(len(preds)) * preds['beat_period_mean'].mean()
if 'beat_period_std' in preds.dtype.names:
    preds['beat_period_std'] = np.ones(len(preds)) * preds['beat_period_std'].mean()

## 4. Decode predictions

Finally, we decode the predictions using a `PerformanceCodec` and export the performance as a MIDI file. The output of the `decode` method is an instance of `partitura.performance.PerformedPart`.

In [8]:
perf_codec = get_performance_codec(model.output_names)
predicted_ppart = perf_codec.decode(part, preds)

Finally, we can export the predictions of the model as a MIDI file!

In [9]:
# export midi
midi_fn = 'my_midi_output.mid'
save_performance_midi(predicted_ppart, midi_fn)